<a href="https://colab.research.google.com/github/Topsan2002/Artificial-Intelligence/blob/main/scrap_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install bs4

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=3f0351e8812f9820dab039878c8464b9edbb14430d9b2a14bc9d623573b1070a
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [ ]:
pip install songline

  Preparing metadata (setup.py) ... done
  Created wheel for songline: filename=songline-0.0.2-py3-none-any.whl size=4868 sha256=eff72be7c3219aa38b95a5ed423eb5410632af9c064646398f4574bd3de4e7a4
  Stored in directory: /root/.cache/pip/wheels/b2/5c/85/9a7ba890449b322545ab4628d052b91155c6379b4f9a8669f4
Successfully built songline


In [ ]:
pip install pandas

In [ ]:
pip install openpyxl

In [ ]:
import pandas as pd
import bs4
import requests
import openpyxl

item = 1
name_list = []
price_list = []

while item <= 69:
  data = requests.get('https://siamintershop.com/category?tskp=' + str(item))
  soup = bs4.BeautifulSoup(data.text)

  name = soup.find_all('div',{'class':'product_name'})
  price = soup.find_all('div',{'class':'product_price'})

  for i in name:
    name_list.append(i.text)
  for i in price:
    price_list.append(i.text)

  print('Complete item number: ',item)
  item += 1
print(name_list)
print(price_list)

table = pd.DataFrame([name_list,price_list]).transpose()
table.columns = ['name','price']
table.set_index('name')
table.to_excel('blob.xlsx',engine='openpyxl')

Complete item number:  1
Complete item number:  2
Complete item number:  3
Complete item number:  4
Complete item number:  5
Complete item number:  6
Complete item number:  7
Complete item number:  8
Complete item number:  9
Complete item number:  10
Complete item number:  11
Complete item number:  12
Complete item number:  13
Complete item number:  14
Complete item number:  15
Complete item number:  16
Complete item number:  17
Complete item number:  18
Complete item number:  19
Complete item number:  20
Complete item number:  21
Complete item number:  22
Complete item number:  23
Complete item number:  24
Complete item number:  25
Complete item number:  26
Complete item number:  27
Complete item number:  28
Complete item number:  29
Complete item number:  30
Complete item number:  31
Complete item number:  32
Complete item number:  33
Complete item number:  34
Complete item number:  35
Complete item number:  36
Complete item number:  37
Complete item number:  38
Complete item number:

In [ ]:
pip install azure-storage-blob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.0/388.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import json

In [ ]:

from azure.storage.blob import BlobServiceClient

#ไปที่ storage account แล้วเลือก access key เพื่อ copy key , account และ conection string ใส่ในช่องว่างแต่ละตัวแปร

storage_account_key = "QU6Ly6cj34BFqssVJ06mxRw2S3Ns1jtKIGMw1gJ4IWMfaw3TrORQDWSYtgzubYKfnMxA64gyJ0Ef+ASt9mieMA=="
storage_account_name = "stbigdatalab03"
connection_string = "DefaultEndpointsProtocol=https;AccountName=stbigdatalab03;AccountKey=QU6Ly6cj34BFqssVJ06mxRw2S3Ns1jtKIGMw1gJ4IWMfaw3TrORQDWSYtgzubYKfnMxA64gyJ0Ef+ASt9mieMA==;EndpointSuffix=core.windows.net"
container_name = "blobinsertdata"

#upload file to blobstorage

def uploadToBlobStorage(file_path,file_name):
  blob_service_client = BlobServiceClient.from_connection_string(connection_string)
  blob_client = blob_service_client.get_blob_client(container=container_name,blob=file_name)
  try:
    if blob_client.exists():
        # Delete blob
        blob_client.delete_blob()
        print("Blob deleted successfully!")
    else:
        print("Blob does not exist!")
    with open(file_path,"rb") as data:
      blob_client.upload_blob(data)
    print(f"Uploaded {file_name}.successfully!")
  except Exception as e:
    print("Failed to upload files to container. Error:" + str(e))

uploadToBlobStorage('/content/blob.xlsx','bloboncontainer.xlsx')

Blob does not exist!
Uploaded bloboncontainer.xlsx.successfully!


In [ ]:
import os
from multiprocessing.pool import ThreadPool
from azure.storage.blob import BlobServiceClient, BlobClient
from azure.storage.blob import ContentSettings, ContainerClient

# IMPORTANT: Replace connection string with your storage account connection string
# Usually starts with DefaultEndpointsProtocol=https;...
MY_CONNECTION_STRING =  "DefaultEndpointsProtocol=https;AccountName=stbigdatalab03;AccountKey=QU6Ly6cj34BFqssVJ06mxRw2S3Ns1jtKIGMw1gJ4IWMfaw3TrORQDWSYtgzubYKfnMxA64gyJ0Ef+ASt9mieMA==;EndpointSuffix=core.windows.net"

# Replace with blob container name
MY_BLOB_CONTAINER = "blobinsertdata"

# Replace with the local folder where you want downloaded files to be stored
LOCAL_BLOB_PATH = "/content/download_data"

class AzureBlobFileDownloader:
  def __init__(self):
    print("Intializing AzureBlobFileDownloader")

    # Initialize the connection to Azure storage account
    self.blob_service_client =  BlobServiceClient.from_connection_string(MY_CONNECTION_STRING)
    self.my_container = self.blob_service_client.get_container_client(MY_BLOB_CONTAINER)

  def download_all_blobs_in_container(self):
    # get a list of blobs
    my_blobs = self.my_container.list_blobs()
    result = self.run(my_blobs)
    print(result)

  def run(self,blobs):
    # Download 10 files at a time!
    with ThreadPool(processes=int(10)) as pool:
     return pool.map(self.save_blob_locally, blobs)

  def save_blob_locally(self,blob):
    file_name = blob.name
    print(file_name)
    bytes = self.my_container.get_blob_client(blob).download_blob().readall()

    # Get full path to the file
    download_file_path = os.path.join(LOCAL_BLOB_PATH, file_name)
    # for nested blobs, create local path as well!
    os.makedirs(os.path.dirname(download_file_path), exist_ok=True)

    with open(download_file_path, "wb") as file:
      file.write(bytes)
    return file_name

# Initialize class and upload files
azure_blob_file_downloader = AzureBlobFileDownloader()
azure_blob_file_downloader.download_all_blobs_in_container()

Intializing AzureBlobFileDownloader
all smartphone.xlsx
blob.xlsx
bloboncontainer.xlsx
stockblob.xlsx
['all smartphone.xlsx', 'blob.xlsx', 'bloboncontainer.xlsx', 'stockblob.xlsx']


In [ ]:
excel_data_df = pd.read_excel('/content/download_data/bloboncontainer.xlsx')
print(excel_data_df)

      Unnamed: 0                                              name       price
0              0                              Vinland Saga เล่ม 12   76.50 บาท
1              1      CHOUJIN X เล่ม 05 (แถมโปสการ์ดลาย Exclusive)   89.25 บาท
2              2                     ดาบหาญกล้าฝ่าแดนยุทธ์ เล่ม 27  238.00 บาท
3              3                                อสูรกลืนภพ เล่ม 16  238.00 บาท
4              4                     อลิซตะลุยแดนมหัศจรรย์ เล่ม 05   85.00 บาท
...          ...                                               ...         ...
3234        3234                                    PPPPPP เล่ม 06   76.50 บาท
3235        3235       คนลึกไขปริสนาลับ BLACK BUTLER เล่ม 11 [NEW]   80.75 บาท
3236        3236  FULLMETAL ALCHEMIST แขนกลคนแปรธาตุ เล่ม 12 [NEW]   76.50 บาท
3237        3237            คุณอาฮาเรน นี่เข้าใจยากจริงๆนะ เล่ม 04   80.75 บาท
3238        3238             RAGNA CRIMSON ตำนานนักล่ามังกร เล่ม 6   80.75 บาท

[3239 rows x 3 columns]
